In [136]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [137]:
# target webpage
url = "https://www.dataiku.com/company/team/"

# Establishing the connection to the web page:
response = requests.get(url)

# You can use status codes to understand how the target server responds to your request.
# Ex., 200 = OK, 400 = Bad Request, 403 = Forbidden, 404 = Not Found.
print(response.status_code)

# Pull the HTML string out of requests and convert it to a Python string.
html = response.text

# The first 700 characters of the content.
#print(html)

200


In [138]:
soup = BeautifulSoup(html, 'lxml')

In [139]:
div = soup.find('div', {'class': 'team-listing'})
div

<div class="team-listing">
<div class="team-card">
<div class="image" style="background-image: url('https://www.dataiku.com/wp-content/uploads/2019/11/Mathieu-A.jpg');"></div>
<div class="info">
<h5>Mathieu A.</h5>
<p>Partner Certification</p>
</div>
</div> <div class="team-card">
<div class="image" style="background-image: url('https://www.dataiku.com/wp-content/uploads/2019/11/Charles-BENETEAU.jpg');"></div>
<div class="info">
<h5>Charles B.</h5>
<p>Regional Director</p>
</div>
</div> <div class="team-card">
<div class="image" style="background-image: url('https://www.dataiku.com/wp-content/uploads/2019/11/Joachim-Z.jpg');"></div>
<div class="info">
<h5>Joachim Z.</h5>
<p>R&amp;D Engineer</p>
</div>
</div> <div class="team-card">
<div class="image" style="background-image: url('https://www.dataiku.com/wp-content/uploads/2019/10/Noha_Badawy-1.jpg');"></div>
<div class="info">
<h5>Noha B.</h5>
<p>Data Scientist</p>
</div>
</div> <div class="team-card">
<div class="image" style="backgro

In [146]:
# creating a list of people
people = []

# iterating through each team-card
for row in div.find_all('div', 'team-card'):
    
    # creating a dictionary for this person
    person = {}
    # their names are listed in the 'h5' tags
    person['name'] = row.find('h5').text
    # their roles are listed in the 'p' tags
    person['role'] = row.find('p')
    
    # adding this peron's records to the list of people
    people.append(person)
    
# creating a dataframe from this list of people
team_listing = pd.DataFrame(people)

In [157]:
# checking number of rows and columns
team_listing.shape

(376, 3)

In [148]:
# checking information on this dataframe
team_listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376 entries, 0 to 375
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    376 non-null    object
 1   role    375 non-null    object
 2   image   0 non-null      object
dtypes: object(3)
memory usage: 8.9+ KB


In [149]:
# checking data types
team_listing.dtypes

name     object
role     object
image    object
dtype: object

In [150]:
team_listing['name'].nunique()

362

In [151]:
team_listing['role'].nunique()

170

In [152]:
# which combinations of (first name, initial of last name) repeat?
team_listing.pivot_table(index=['name'], aggfunc='size').sort_values(ascending = False).head(14)

name
Kyle A.          2
Christiaan B.    2
Terry O.         2
Emilie C.        2
Nicolas O.       2
Tara C.          2
Ned M.           2
Liam S.          2
Aurélien V.      2
Cooper J.        2
Mike M.          2
Deborah G.       2
Ina J.           2
Louis B.         2
dtype: int64

In [156]:
# looks like Nicholas O. and Emilie C. are simply people with the same names, but in different roles. 
team_listing[team_listing.duplicated()]

,name,role,image
96,Deborah G.,[Senior Account Executive],None
111,Ina J.,[Marketing Enablement Specialist],None
138,Terry O.,[Corporate Controller],None
205,Louis B.,[Sales Development Representative],None
280,Christiaan B.,[Business Development Specialist],None
300,Cooper J.,[Business Development Specialist],None
322,Mike M.,[Account Executive],None
339,Liam S.,[Business Development Specialist],None
350,Aurélien V.,[Implementation Manager],None
355,Ned M.,[DSS Training Specialist],None


I have identified these 12 individuals who have their records duplicated on the team website: https://www.dataiku.com/company/team/ with the only difference being that one of their photos is a zoomed in version of the other, in no particular order. In some cases, the first instance is zoomed in, in others it is the second instance. 

In [159]:
# full list
team_listing.to_csv('./data/full_team_list.csv')

In [160]:
# duplicated list
team_listing[team_listing.duplicated()].to_csv('./data/duplicated_list.csv')